# Importing needed packages

In [1]:
import re
import unicodedata

import pandas as pd
import requests
from bs4 import BeautifulSoup

# Parsing main page

Getting the code for the page with movie

In [8]:
f = requests.get("https://www.kinopoisk.ru/film/339/")

Passing the code into the `BeautifulSoup` constructor

In [9]:
soup = BeautifulSoup(f.text, "html.parser")

In [11]:
movie_dict = {}

## Parsing movie's name

In [12]:
movie_name_dict = {}

In [13]:
movie_name = soup.find_all("h1", attrs={"class": re.compile("styles_title")})[0]

In [14]:
movie_name.get_text()

'Индиана Джонс: В поисках утраченного ковчега (1981)'

In [15]:
movie_name_original = soup.find_all(
    "span", attrs={"class": re.compile("styles_originalTitle")}
)[0]

In [16]:
movie_name_original.get_text()

'Raiders of the Lost Ark'

In [17]:
movie_name_dict["Название"] = movie_name.get_text()
movie_name_dict["Оргинальное название"] = movie_name_original.get_text()

## Parsing information about the movie

In [18]:
movie_info_dict = {}

In [19]:
movie_info_divs = soup.find_all(attrs={"data-test-id": "encyclopedic-table"})[0]

In [20]:
values = []
for div in movie_info_divs.find_all(
    "div", attrs={"class": re.compile("styles_value"), "data-tid": re.compile(".*")}
):
    current_value = div.get_text().replace("сборы", "")
    current_value = unicodedata.normalize("NFKD", current_value)
    if "слова" not in current_value:
        values.append(current_value)

In [21]:
titles = []
for div in movie_info_divs.find_all(class_=re.compile("styles_title")):
    current_title = div.get_text()
    titles.append(current_title)

In [22]:
assert len(values) == len(titles)
movie_info_dict = dict(zip(titles, values))
movie_dict = {**movie_name_dict, **movie_info_dict}

In [23]:
movie_dict

{'Название': 'Индиана Джонс: В поисках утраченного ковчега (1981)',
 'Оргинальное название': 'Raiders of the Lost Ark',
 'Год производства': '1981',
 'Страна': 'США',
 'Жанр': 'приключения, боевик, комедия',
 'Слоган': '«Indiana Jones - the new hero from the creators of JAWS and STAR WARS»',
 'Режиссер': 'Стивен Спилберг',
 'Сценарий': 'Лоуренс Кэздан, Джордж Лукас, Филип Кауфман',
 'Продюсер': 'Ховард Дж. Казанян, Джордж Лукас, Фрэнк Маршалл, ...',
 'Оператор': 'Дуглас Слоком',
 'Композитор': 'Джон Уильямс',
 'Художник': 'Норман Рейнольдс, Айвор Лесли Дилли, Дебора Нэдулмэн, ...',
 'Монтаж': 'Майкл Кан, Джордж Лукас',
 'Бюджет': '$18 000 000',
 'Сборы в США': '$212 222 025',
 'Сборы в мире': '+ $141 766 000 = $353 988 025',
 'Зрители': '88.1 млн , 6.4 млн , 4.2 млн , ...',
 'Премьера в мире': '12 июня 1981, ...',
 'Релиз на DVD': '20 ноября 2003, «Premier Digital»',
 'Возраст': '12+',
 'Рейтинг MPAA': 'PGрекомендуется присутствие родителей',
 'Время': '115 мин. / 01:55'}

## Parsing movie's rating

In [ ]:
movie_rating_dict = {}

In [ ]:
movie_name = soup.find_all("h1", attrs={"class": re.compile("styles_title")})[0]

In [ ]:
movie_name.get_text()

In [ ]:
movie_name_original = soup.find_all(
    "span", attrs={"class": re.compile("styles_originalTitle")}
)[0]

In [ ]:
movie_name_original.get_text()

In [ ]:
movie_name_dict["Название"] = movie_name.get_text()
movie_name_dict["Оргинальное название"] = movie_name_original.get_text()